# Assignment-01

## 1. Implementation of the Apri-ori and the rule extractor al-gorithms (8 Pts.)


In [11]:
# your code here 

## 2. Mining frequent itemsets and association rules in a movie rating dataset (22Pts.)

### 2.1 Building the transactionaldatabases (8 pts.)

In [23]:
def create_transaction():
    with open('MovieLensDatasets/ratings.dat', 'r') as file:
        data = file.readlines()
        positive_transactions = {}
        negative_transactions = {}
        for row in data:
            user_id, movie_id, rating, _ = map(int, row.strip().split('::'))
            if user_id not in positive_transactions:
                positive_transactions[user_id] = []
            if user_id not in negative_transactions:
                negative_transactions[user_id] = []
            if rating >= 4:
                positive_transactions[user_id].append(movie_id)
            elif rating <= 2:
                negative_transactions[user_id].append(movie_id)
    return positive_transactions, negative_transactions

positive_transactions, negative_transactions = create_transaction()

### 2.2 Getting first results (6 pts.)

In [28]:
def create_movie_name_dict():
    with open('MovieLensDatasets/movies.dat', 'r', encoding='latin-1') as file:
        movie_names = {}
        for row in file:
            parts = row.rstrip('\n').split('::', 2)
            movie_id = int(parts[0])
            name = parts[1]
            movie_names[movie_id] = name
    return movie_names

movie_name_dict = create_movie_name_dict()
print(movie_name_dict[positive_transactions[1][1]])


Erin Brockovich (2000)


### 2.3 A deeper look (8 Pts.)

In [ ]:
# your code here